In [1]:
import gdspy
import numpy as np

Load the gds file and create a target file to save edited design

In [2]:
gdsii = gdspy.GdsLibrary()
file = 'design_final1.gds'
#read the design file to meet compliance
gdsii.read_gds(file, units = 'import')

#create a new file 
gds_new = gdspy.GdsLibrary(unit = 1e-6, precision = 5e-10)

create a new cell to put the gds design

In [3]:
top = gds_new.new_cell('TOP')

Specify layer numbers

In [4]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60
chip_edge_layer = 703
text_layer1 = 98

layer_numbers  = [qubit_layer,junction_layer,ab_layer,ab_square_layer,junction_area_layer, chip_edge_layer]

specify target datatype

In [5]:
round(0.25)

0

In [6]:
data_type = 0

In [7]:
#snap to grid function that meets the layer requirements 
def snap_to_grid(coord, grid_size):
    new = (coord/grid_size).astype(int)
    res = []
    for (x,y) in new:
        if x > 0 and y>0:
            pass
        elif x > 0 and y<0:
            y -= 1
        elif x < 0 and y>0:
            x -= 1
        else:
            x -= 1
            y -= 1
        res += [[x * grid_size, y*grid_size]]
    return np.array(res)

In [8]:
qubit_layer_grid = 50*1e-6
junction_layer_grid = 5*1e-6

Get the polygons from the old design

In [9]:
polys = gdsii.top_level()[0].get_polygons(by_spec = True)

In [10]:
len(polys)

7

In [11]:
keys = list(polys.keys())

In [12]:
keys

[(5, 0), (20, 0), (31, 0), (30, 0), (60, 0), (703, 0), (98, 0)]

Extract all the data types for a given layer number

In [13]:
datatypes = {}
for i in keys:
    try:
        print(datatypes[i[0]])
    except:
        datatypes[i[0]] = [i[1]]
    else:
        datatypes[i[0]].append(i[1])


In [14]:
datatypes

{5: [0], 20: [0], 31: [0], 30: [0], 60: [0], 703: [0], 98: [0]}

enumerate over layer numbers and save the polygon into the new design with desired layer number

In [15]:
count = 0
for i, layer in enumerate(layer_numbers):

    for j in datatypes[layer]:
        source_polyons = polys.get((layer,j),[])

        for polygon in source_polyons:
            if (layer ==5) or (layer == 31) or (layer == 30):
                polygon = snap_to_grid(polygon, qubit_layer_grid)
            elif (layer == 20) or (layer == 60):
                polygon = snap_to_grid(polygon, junction_layer_grid)
            if layer == 5:
                pass
            else:
                polygon = polygon[:-1]
                
            maxx = np.max(polygon[:,0])
            minx = np.min(polygon[:,0])
            maxy = np.max(polygon[:,1])
            miny = np.min(polygon[:,1])
            
            # if ((maxx-minx) >= 1e3*10.15) or ((maxy-miny) >= 1e3*10.15):
            #     continue
            p =gdspy.Polygon(polygon*(1e-3/1e-6), layer = int(layer), datatype = 0)
            # print(cell_names[i])
            count+=1
            
            
            # top.add(p)

In [16]:
count

58191

add the chip edge

In [17]:
rectangle = gdspy.Polygon(1e3*np.array([[-10.15/2,10.15/2],[10.15/2,10.15/2],[10.15/2,-10.15/2],[-10.15/2,-10.15/2]]), layer = int(chip_edge_layer))
top.add(rectangle)

In [18]:
rectangle

Write necessary information in the information cells

In [19]:
htest1 = gdspy.Text('Email',1/2*1000,(-5000,4000), layer = text_layer1)
htest2 = gdspy.Text('wendywan@stanford.edu',1/2*1000, (-5000,2000), layer = text_layer1)
htest3 = gdspy.Text('Jc = 0.1uA/um2',1/2*1000,(-5000,0), layer = text_layer1)

In [20]:
top.add(htest1)
top.add(htest2)
top.add(htest3)

In [21]:
gds_new.write_gds('design_final1.gds')